# Predicting CONCACAF WCQ table

In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dataframe_image as dfi

In [82]:
current_CONCACAF_table = pd.read_csv('./data/current_table_12.14.2021.csv')
current_CONCACAF_table

,rank,team_name,matches_played,wins,draws,losses,goals_for,goals_against,goal_differential,points
0,1,Canada,8,4,4,0,13,5,8,16
1,2,USA,8,4,3,1,12,5,7,15
2,3,Mexico,8,4,2,2,11,7,4,14
3,4,Panama,8,4,2,2,11,9,2,14
4,5,Costa Rica,8,2,3,3,6,7,-1,9
5,6,Jamaica,8,1,4,3,6,10,-4,7
6,7,El Salvador,8,1,3,4,4,10,-6,6
7,8,Honduras,8,0,3,5,5,15,-10,3


In [83]:
dfi.export(current_CONCACAF_table, './images/current_table.png')

In [84]:
# creating new table for predictions

predictions_dict = {'team_name': ['Canada', 'United States', 'Mexico', 'Panama', 'Costa Rica', 'Jamaica', 'El Salvador', 'Honduras'], 
'matches_played': [0, 0, 0, 0, 0, 0, 0, 0], 'wins': [0, 0, 0, 0, 0, 0, 0, 0], 'draws': [0, 0, 0, 0, 0, 0, 0, 0],
'losses': [0, 0, 0, 0, 0, 0, 0, 0], 'goals_for': [0, 0, 0, 0, 0, 0, 0, 0], 'goals_against': [0, 0, 0, 0, 0, 0, 0, 0], 
'goal_differential': [0, 0, 0, 0, 0, 0, 0, 0], 'points': [0, 0, 0, 0, 0, 0, 0, 0]}

predicted_table = pd.DataFrame(predictions_dict)
predicted_table

,team_name,matches_played,wins,draws,losses,goals_for,goals_against,goal_differential,points
0,Canada,0,0,0,0,0,0,0,0
1,United States,0,0,0,0,0,0,0,0
2,Mexico,0,0,0,0,0,0,0,0
3,Panama,0,0,0,0,0,0,0,0
4,Costa Rica,0,0,0,0,0,0,0,0
5,Jamaica,0,0,0,0,0,0,0,0
6,El Salvador,0,0,0,0,0,0,0,0
7,Honduras,0,0,0,0,0,0,0,0


In [85]:
qualifying_predictions = pd.read_csv('./data/qualifier_predictions.csv')
# qualifying_fixtures

In [86]:
qualifying_predictions_dict = qualifying_predictions.to_dict('records')
# qualifying_fixtures_dict[0]

predicted_table_dict = predicted_table.to_dict('records')
# predicted_table_dict[0]

In [87]:
# Updating predictions table with predictions for qualifying matches

for i in qualifying_predictions_dict:
    for j in predicted_table_dict:
        if i['home_team'] == j['team_name']:
            j['matches_played'] = j['matches_played'] + 1
            j['goals_for'] = j['goals_for'] + i['home_score']
            j['goals_against'] = j['goals_against'] + i['away_score']
            j['goal_differential'] = j['goal_differential'] + i['home_score'] - i['away_score']
            if i['home_score'] > i['away_score']:
                j['wins'] = j['wins'] + 1
                j['points'] = j['points'] + 3
            elif i['home_score'] == i['away_score']:
                j['draws'] = j['draws'] + 1
                j['points'] = j['points'] + 1
            elif i['home_score'] < i['away_score']:
                j['losses'] = j['losses'] + 1
    
# pd.DataFrame(predicted_table_dict)

for i in qualifying_predictions_dict:
    for j in predicted_table_dict:
        if i['away_team'] == j['team_name']:
            j['matches_played'] = j['matches_played'] + 1
            j['goals_for'] = j['goals_for'] + i['away_score']
            j['goals_against'] = j['goals_against'] + i['home_score']
            j['goal_differential'] = j['goal_differential'] + i['away_score'] - i['home_score']
            if i['away_score'] > i['home_score']:
                j['wins'] = j['wins'] + 1
                j['points'] = j['points'] + 3
            elif i['away_score'] == i['home_score']:
                j['draws'] = j['draws'] + 1
                j['points'] = j['points'] + 1
            elif i['away_score'] < i['home_score']:
                j['losses'] = j['losses'] + 1
    
predicted_table = pd.DataFrame(predicted_table_dict)
predicted_table

,team_name,matches_played,wins,draws,losses,goals_for,goals_against,goal_differential,points
0,Canada,14,3,7,4,13,15,-2,16
1,United States,14,10,4,0,28,6,22,34
2,Mexico,14,12,2,0,33,4,29,38
3,Panama,14,0,4,10,8,27,-19,4
4,Costa Rica,14,2,9,3,14,17,-3,15
5,Jamaica,14,4,7,3,15,16,-1,19
6,El Salvador,14,2,5,7,8,19,-11,11
7,Honduras,14,2,4,8,8,23,-15,10


In [88]:
predicted_table = predicted_table.reindex(predicted_table.sort_values(['points', 'goal_differential', 'goals_for'], ascending=[False, False, False]).index)
predicted_table.reset_index(inplace=True)

predicted_table.rename(columns={'index': 'rank'}, inplace=True)
predicted_table['rank'] = list(range(1,9))
predicted_table.set_index(predicted_table['rank'], inplace=True)
predicted_table.drop(columns=['rank'], inplace=True)
predicted_table

,team_name,matches_played,wins,draws,losses,goals_for,goals_against,goal_differential,points
rank,,,,,,,,,
1,Mexico,14,12,2,0,33,4,29,38
2,United States,14,10,4,0,28,6,22,34
3,Jamaica,14,4,7,3,15,16,-1,19
4,Canada,14,3,7,4,13,15,-2,16
5,Costa Rica,14,2,9,3,14,17,-3,15
6,El Salvador,14,2,5,7,8,19,-11,11
7,Honduras,14,2,4,8,8,23,-15,10
8,Panama,14,0,4,10,8,27,-19,4


In [89]:
dfi.export(predicted_table, './images/predicted_table.png')

---

# Checking predictions against qualifier results

### All predictions

In [90]:
qualifying_fixtures = pd.read_csv('./data/concacaf_wcq_fixtures.csv')
qualifying_fixtures = qualifying_fixtures[['date', 'home_team', 'home_score', 'away_score', 'away_team']].head(32)
qualifying_fixtures['home_score'] = qualifying_fixtures['home_score'].astype(int)
qualifying_fixtures['away_score'] = qualifying_fixtures['away_score'].astype(int)
qualifying_fixtures

,date,home_team,home_score,away_score,away_team
0,2021-09-02,Canada,1,1,Honduras
1,2021-09-02,Panama,0,0,Costa Rica
2,2021-09-02,Mexico,2,1,Jamaica
3,2021-09-02,El Salvador,0,0,United States
4,2021-09-05,Jamaica,0,3,Panama
5,2021-09-05,El Salvador,0,0,Honduras
6,2021-09-05,Costa Rica,0,1,Mexico
7,2021-09-05,United States,1,1,Canada
8,2021-09-08,Canada,3,0,El Salvador
9,2021-09-08,Panama,1,1,Mexico


In [91]:
qualifying_predictions = qualifying_predictions[['home_team', 'home_score', 'away_score', 'away_team', 'date']]
qualifying_predictions.rename(columns={'home_team': 'preds_home_team', 'home_score': 'preds_home_score',
                                      'away_score': 'preds_away_score', 'away_team': 'preds_away_team', 
                                       'date': 'preds_date'}, inplace=True)

In [92]:
real_vs_pred = pd.concat([qualifying_fixtures, qualifying_predictions], axis=1)
real_vs_pred = real_vs_pred.head(32)
real_vs_pred

,date,home_team,home_score,away_score,away_team,preds_home_team,preds_home_score,preds_away_score,preds_away_team,preds_date
0,2021-09-02,Canada,1.0,1.0,Honduras,Canada,2,0,Honduras,2021-09-02
1,2021-09-02,Panama,0.0,0.0,Costa Rica,Panama,1,1,Costa Rica,2021-09-02
2,2021-09-02,Mexico,2.0,1.0,Jamaica,Mexico,3,0,Jamaica,2021-09-02
3,2021-09-02,El Salvador,0.0,0.0,United States,El Salvador,0,2,United States,2021-09-02
4,2021-09-05,Jamaica,0.0,3.0,Panama,Jamaica,2,0,Panama,2021-09-05
5,2021-09-05,El Salvador,0.0,0.0,Honduras,El Salvador,2,0,Honduras,2021-09-05
6,2021-09-05,Costa Rica,0.0,1.0,Mexico,Costa Rica,1,2,Mexico,2021-09-05
7,2021-09-05,United States,1.0,1.0,Canada,United States,2,0,Canada,2021-09-05
8,2021-09-08,Canada,3.0,0.0,El Salvador,Canada,2,0,El Salvador,2021-09-08
9,2021-09-08,Panama,1.0,1.0,Mexico,Panama,1,3,Mexico,2021-09-08


In [93]:
dfi.export(real_vs_pred, './images/actual_vs_predicted.png')

In [94]:
# 6 correct home and away score predictions in 32 matches (18.75%)
# 17 correct predictions of outcome in 32 matches (53.13%)
# 12 correct predictions of home team score (37.50%)
# 11 correct predictions of away team score (34.38%)
# 12 correct predictions of total goals scored (37.50%)

---

## Predictions by team

### Mexico

In [104]:
real_vs_pred[(real_vs_pred['home_team'] == 'Mexico') | (real_vs_pred['away_team'] == 'Mexico')]

,date,home_team,home_score,away_score,away_team,preds_home_team,preds_home_score,preds_away_score,preds_away_team,preds_date
2,2021-09-02,Mexico,2.0,1.0,Jamaica,Mexico,3,0,Jamaica,2021-09-02
6,2021-09-05,Costa Rica,0.0,1.0,Mexico,Costa Rica,1,2,Mexico,2021-09-05
9,2021-09-08,Panama,1.0,1.0,Mexico,Panama,1,3,Mexico,2021-09-08
14,2021-10-07,Mexico,1.0,1.0,Canada,Mexico,2,0,Canada,2021-10-07
19,2021-10-10,Mexico,3.0,0.0,Honduras,Mexico,4,0,Honduras,2021-10-10
23,2021-10-13,El Salvador,0.0,2.0,Mexico,El Salvador,0,2,Mexico,2021-10-13
24,2021-11-11,United States,2.0,0.0,Mexico,United States,1,1,Mexico,2021-11-11
31,2021-11-16,Canada,2.0,1.0,Mexico,Canada,0,2,Mexico,2021-11-16


In [105]:
# 1 correct home and away score predictions in (12.5%)
# 4 correct predictions of outcome in (50%)
# 1 correct predictions of home team score (12.5%)
# 2 correct predictions of away team score (25%)
# 4 correct predictions of total goals scored (50%)

---

### United States

In [106]:
real_vs_pred[(real_vs_pred['home_team'] == 'United States') | (real_vs_pred['away_team'] == 'United States')]

,date,home_team,home_score,away_score,away_team,preds_home_team,preds_home_score,preds_away_score,preds_away_team,preds_date
3,2021-09-02,El Salvador,0.0,0.0,United States,El Salvador,0,2,United States,2021-09-02
7,2021-09-05,United States,1.0,1.0,Canada,United States,2,0,Canada,2021-09-05
11,2021-09-08,Honduras,1.0,4.0,United States,Honduras,0,2,United States,2021-09-08
13,2021-10-07,United States,2.0,0.0,Jamaica,United States,3,1,Jamaica,2021-10-07
17,2021-10-10,Panama,1.0,0.0,United States,Panama,1,2,United States,2021-10-10
20,2021-10-12,United States,2.0,1.0,Costa Rica,United States,2,0,Costa Rica,2021-10-12
24,2021-11-11,United States,2.0,0.0,Mexico,United States,1,1,Mexico,2021-11-11
28,2021-11-16,Jamaica,1.0,1.0,United States,Jamaica,1,1,United States,2021-11-16


In [107]:
# 1 correct home and away score predictions in (12.5%)
# 4 correct predictions of outcome in (50%)
# 4 correct predictions of home team score (50%)
# 1 correct predictions of away team score (12.5%)
# 3 correct predictions of total goals scored (37.5%)

---

### Jamaica

In [108]:
real_vs_pred[(real_vs_pred['home_team'] == 'Jamaica') | (real_vs_pred['away_team'] == 'Jamaica')]

,date,home_team,home_score,away_score,away_team,preds_home_team,preds_home_score,preds_away_score,preds_away_team,preds_date
2,2021-09-02,Mexico,2.0,1.0,Jamaica,Mexico,3,0,Jamaica,2021-09-02
4,2021-09-05,Jamaica,0.0,3.0,Panama,Jamaica,2,0,Panama,2021-09-05
10,2021-09-08,Costa Rica,1.0,1.0,Jamaica,Costa Rica,1,1,Jamaica,2021-09-08
13,2021-10-07,United States,2.0,0.0,Jamaica,United States,3,1,Jamaica,2021-10-07
16,2021-10-10,Jamaica,0.0,0.0,Canada,Jamaica,1,1,Canada,2021-10-10
22,2021-10-13,Honduras,0.0,2.0,Jamaica,Honduras,1,1,Jamaica,2021-10-13
26,2021-11-12,El Salvador,1.0,1.0,Jamaica,El Salvador,1,1,Jamaica,2021-11-12
28,2021-11-16,Jamaica,1.0,1.0,United States,Jamaica,1,1,United States,2021-11-16


In [109]:
# 3 correct home and away score predictions in (37.5%)
# 6 correct predictions of outcome in (75%)
# 3 correct predictions of home team score (37.5%)
# 3 correct predictions of away team score (37.5%)
# 5 correct predictions of total goals scored (62.5%)

---

### Canada

In [98]:
real_vs_pred[(real_vs_pred['home_team'] == 'Canada') | (real_vs_pred['away_team'] == 'Canada')]

,date,home_team,home_score,away_score,away_team,preds_home_team,preds_home_score,preds_away_score,preds_away_team,preds_date
0,2021-09-02,Canada,1.0,1.0,Honduras,Canada,2,0,Honduras,2021-09-02
7,2021-09-05,United States,1.0,1.0,Canada,United States,2,0,Canada,2021-09-05
8,2021-09-08,Canada,3.0,0.0,El Salvador,Canada,2,0,El Salvador,2021-09-08
14,2021-10-07,Mexico,1.0,1.0,Canada,Mexico,2,0,Canada,2021-10-07
16,2021-10-10,Jamaica,0.0,0.0,Canada,Jamaica,1,1,Canada,2021-10-10
21,2021-10-13,Canada,4.0,1.0,Panama,Canada,2,0,Panama,2021-10-13
27,2021-11-12,Canada,1.0,0.0,Costa Rica,Canada,1,1,Costa Rica,2021-11-12
31,2021-11-16,Canada,2.0,1.0,Mexico,Canada,0,2,Mexico,2021-11-16


In [110]:
# 0 correct home and away score predictions in (0%)
# 3 correct predictions of outcome in (37.5%)
# 1 correct predictions of home team score (12.5%)
# 1 correct predictions of away team score (12.5%)
# 3 correct predictions of total goals scored (37.5%)

---

### Costa Rica

In [99]:
real_vs_pred[(real_vs_pred['home_team'] == 'Costa Rica') | (real_vs_pred['away_team'] == 'Costa Rica')]

,date,home_team,home_score,away_score,away_team,preds_home_team,preds_home_score,preds_away_score,preds_away_team,preds_date
1,2021-09-02,Panama,0.0,0.0,Costa Rica,Panama,1,1,Costa Rica,2021-09-02
6,2021-09-05,Costa Rica,0.0,1.0,Mexico,Costa Rica,1,2,Mexico,2021-09-05
10,2021-09-08,Costa Rica,1.0,1.0,Jamaica,Costa Rica,1,1,Jamaica,2021-09-08
12,2021-10-07,Honduras,0.0,0.0,Costa Rica,Honduras,1,1,Costa Rica,2021-10-07
18,2021-10-10,Costa Rica,2.0,1.0,El Salvador,Costa Rica,1,1,El Salvador,2021-10-10
20,2021-10-12,United States,2.0,1.0,Costa Rica,United States,2,0,Costa Rica,2021-10-12
27,2021-11-12,Canada,1.0,0.0,Costa Rica,Canada,1,1,Costa Rica,2021-11-12
30,2021-11-16,Costa Rica,2.0,1.0,Honduras,Costa Rica,2,1,Honduras,2021-11-16


In [111]:
# 2 correct home and away score predictions in (25%)
# 6 correct predictions of outcome in (75%)
# 4 correct predictions of home team score (50%)
# 3 correct predictions of away team score (37.5%)
# 2 correct predictions of total goals scored (25%)

---

### El Salvador

In [100]:
real_vs_pred[(real_vs_pred['home_team'] == 'El Salvador') | (real_vs_pred['away_team'] == 'El Salvador')]

,date,home_team,home_score,away_score,away_team,preds_home_team,preds_home_score,preds_away_score,preds_away_team,preds_date
3,2021-09-02,El Salvador,0.0,0.0,United States,El Salvador,0,2,United States,2021-09-02
5,2021-09-05,El Salvador,0.0,0.0,Honduras,El Salvador,2,0,Honduras,2021-09-05
8,2021-09-08,Canada,3.0,0.0,El Salvador,Canada,2,0,El Salvador,2021-09-08
15,2021-10-07,El Salvador,1.0,0.0,Panama,El Salvador,1,0,Panama,2021-10-07
18,2021-10-10,Costa Rica,2.0,1.0,El Salvador,Costa Rica,1,1,El Salvador,2021-10-10
23,2021-10-13,El Salvador,0.0,2.0,Mexico,El Salvador,0,2,Mexico,2021-10-13
26,2021-11-12,El Salvador,1.0,1.0,Jamaica,El Salvador,1,1,Jamaica,2021-11-12
29,2021-11-16,Panama,2.0,1.0,El Salvador,Panama,1,1,El Salvador,2021-11-16


In [112]:
# 3 correct home and away score predictions in (37.5%)
# 4 correct predictions of outcome in (50%)
# 4 correct predictions of home team score (50%)
# 7 correct predictions of away team score (87.5%)
# 3 correct predictions of total goals scored (37.5%)

---

### Honduras

In [101]:
real_vs_pred[(real_vs_pred['home_team'] == 'Honduras') | (real_vs_pred['away_team'] == 'Honduras')]

,date,home_team,home_score,away_score,away_team,preds_home_team,preds_home_score,preds_away_score,preds_away_team,preds_date
0,2021-09-02,Canada,1.0,1.0,Honduras,Canada,2,0,Honduras,2021-09-02
5,2021-09-05,El Salvador,0.0,0.0,Honduras,El Salvador,2,0,Honduras,2021-09-05
11,2021-09-08,Honduras,1.0,4.0,United States,Honduras,0,2,United States,2021-09-08
12,2021-10-07,Honduras,0.0,0.0,Costa Rica,Honduras,1,1,Costa Rica,2021-10-07
19,2021-10-10,Mexico,3.0,0.0,Honduras,Mexico,4,0,Honduras,2021-10-10
22,2021-10-13,Honduras,0.0,2.0,Jamaica,Honduras,1,1,Jamaica,2021-10-13
25,2021-11-12,Honduras,2.0,3.0,Panama,Honduras,2,0,Panama,2021-11-12
30,2021-11-16,Costa Rica,2.0,1.0,Honduras,Costa Rica,2,1,Honduras,2021-11-16


In [113]:
# 1 correct home and away score predictions in (12.5%)
# 4 correct predictions of outcome in (50%)
# 2 correct predictions of home team score (25%)
# 3 correct predictions of away team score (37.5%)
# 3 correct predictions of total goals scored (37.5%)

---

### Panama

In [103]:
real_vs_pred[(real_vs_pred['home_team'] == 'Panama') | (real_vs_pred['away_team'] == 'Panama')]

,date,home_team,home_score,away_score,away_team,preds_home_team,preds_home_score,preds_away_score,preds_away_team,preds_date
1,2021-09-02,Panama,0.0,0.0,Costa Rica,Panama,1,1,Costa Rica,2021-09-02
4,2021-09-05,Jamaica,0.0,3.0,Panama,Jamaica,2,0,Panama,2021-09-05
9,2021-09-08,Panama,1.0,1.0,Mexico,Panama,1,3,Mexico,2021-09-08
15,2021-10-07,El Salvador,1.0,0.0,Panama,El Salvador,1,0,Panama,2021-10-07
17,2021-10-10,Panama,1.0,0.0,United States,Panama,1,2,United States,2021-10-10
21,2021-10-13,Canada,4.0,1.0,Panama,Canada,2,0,Panama,2021-10-13
25,2021-11-12,Honduras,2.0,3.0,Panama,Honduras,2,0,Panama,2021-11-12
29,2021-11-16,Panama,2.0,1.0,El Salvador,Panama,1,1,El Salvador,2021-11-16


In [114]:
# 1 correct home and away score predictions (12.5%)
# 3 correct predictions of outcome (37.5%)
# 4 correct predictions of home team score (50%)
# 2 correct predictions of away team score (25%)
# 1 correct predictions of total goals scored (12.5%)

---